In [54]:
import pandas as pd
import geopandas as gpd
from datetime import datetime
import json
import numpy as np
import locale
import os
import pytz
# Imposta la lingua italiana
locale.setlocale(locale.LC_TIME, 'it_IT.UTF-8')

'it_IT.UTF-8'

In [55]:
PARKS_GEOPARQUET = "data" + os.sep + "parks.geoparquet"
ZONES_GEOPARQUET = "data" + os.sep + "zones.geoparquet"


In [56]:
parks = gpd.read_parquet(PARKS_GEOPARQUET)
zones = gpd.read_parquet(ZONES_GEOPARQUET)

In [57]:
descriptions = {}

In [58]:
zones.columns

Index(['id', 'driver', 'geom', 'name', 'type', 'city', 'color', 'capacity',
       'freeslots', 'ts', 'stall_blu_capacity', 'stall_blu_freeslots',
       'stall_blu_ts', 'stall_carico-scarico_capacity',
       'stall_carico-scarico_freeslots', 'stall_carico-scarico_ts',
       'stall_disabili_capacity', 'stall_disabili_freeslots',
       'stall_disabili_ts'],
      dtype='object')

In [59]:
# Assuming parks.currentTimestamp contains the timestamp
timestamp_parks = parks.currentTimestamp.max()
lastime_parks = timestamp_parks.strftime('%d/%m/%Y %H:%M')
lastime_parks = timestamp_parks.strftime('%d %B %Y ore %H:%M')

In [60]:
# Se il timestamp non è timezone-aware, specifica che è UTC (o il fuso corretto)
if timestamp_parks.tzinfo is None:
    timestamp_parks = timestamp_parks.tz_localize('UTC')  # Cambia se il timestamp è già in un altro fuso

# Converti in fuso orario Europe/Rome
rome_tz = pytz.timezone('Europe/Rome')
timestamp_parks = timestamp_parks.astimezone(rome_tz)

# Formatta la stringa con il fuso orario corretto
lastime_parks = timestamp_parks.strftime('%d %B %Y ore %H:%M')

In [61]:
lastime_parks

'12 marzo 2025 ore 17:10'

In [62]:
descriptions['timestamp_parks'] = lastime_parks

In [63]:
# Assuming parks.currentTimestamp contains the timestamp
timestamp_zones = zones.ts.max()
lastime_zones = timestamp_zones.strftime('%d/%m/%Y %H:%M')

In [ ]:
total_carparkspaces = parks[(parks['type'] == 'park') & (parks.currentTimestamp == parks.currentTimestamp.max())].capacity.sum()
total_carparkspaces_free = parks[(parks['type'] == 'park') & (parks.currentTimestamp == parks.currentTimestamp.max())].freeslots.sum()
total_bikeparkspaces = parks[(parks['type'] == 'bike') & (parks.currentTimestamp == parks.currentTimestamp.max())].capacity.sum()
total_bikeparkspaces_free = parks[(parks['type'] == 'bike') & (parks.currentTimestamp == parks.currentTimestamp.max())].freeslots.sum()

In [65]:
descriptions["total_carparkspaces"] = total_carparkspaces
descriptions["total_carparkspaces_free"] = total_carparkspaces_free
descriptions["total_bikespaces"] = total_bikeparkspaces
descriptions["total_bikespaces_free"] = total_bikeparkspaces_free

In [66]:
to_int = ['stall_blu_capacity','stall_blu_freeslots',
          'stall_carico-scarico_capacity','stall_carico-scarico_freeslots',
          'stall_disabili_capacity','stall_disabili_freeslots']
zones[to_int] = zones[to_int].astype(int)

In [67]:
total_zonespaces = zones[zones.ts == timestamp_zones].capacity.sum()
total_zonespaces_free = zones[zones.ts == timestamp_zones].freeslots.sum()
total_zonespaces_blu = zones[zones.ts == timestamp_zones].stall_blu_capacity.sum()
total_zonespaces_blu_free = zones[zones.ts == timestamp_zones].stall_blu_freeslots.sum()
total_zonespaces_carico_scarico = zones[zones.ts == timestamp_zones]['stall_carico-scarico_capacity'].sum()
total_zonespaces_carico_scarico_free = zones[zones.ts == timestamp_zones]['stall_carico-scarico_freeslots'].sum()
total_zonespaces_disabili = zones[zones.ts == timestamp_zones].stall_disabili_capacity.sum()
total_zonespaces_disabili_free = zones[zones.ts == timestamp_zones].stall_disabili_freeslots.sum()

In [68]:
descriptions["total_zonespaces"] = total_zonespaces
descriptions["total_zonespaces_free"] = total_zonespaces_free
descriptions["total_zonespaces_blu"] = total_zonespaces_blu
descriptions["total_zonespaces_blu_free"] = total_zonespaces_blu_free
descriptions["total_zonespaces_carico_scarico"] = total_zonespaces_carico_scarico
descriptions["total_zonespaces_carico_scarico_free"] = total_zonespaces_carico_scarico_free
descriptions["total_zonespaces_disabili"] = total_zonespaces_disabili
descriptions["total_zonespaces_disabili_free"] = total_zonespaces_disabili_free

In [69]:
total_parks = len(parks[parks['type'] == 'park'].name.unique())
total_ciclobox = len(parks[parks['type'] == 'bike'].name.unique())
total_zones = len(zones.name.unique())

In [70]:
descriptions["total_parks"] = total_parks
descriptions["total_ciclobox"] = total_ciclobox
descriptions["total_zones"] = total_zones

In [71]:
zones[zones.stall_disabili_capacity > 0].name.unique()
zones[zones["stall_carico-scarico_capacity"] > 0].name.unique()

array(['Area di 1a corona'], dtype=object)

In [72]:
descriptions = {k: int(v) if isinstance(v, np.integer) else v for k, v in descriptions.items()}

In [74]:
descriptions

{'timestamp_parks': '12 marzo 2025 ore 17:10',
 'total_carparkspaces': 0,
 'total_carparkspaces_free': 0,
 'total_bikespaces': 0,
 'total_bikespaces_free': 0,
 'total_zonespaces': 285,
 'total_zonespaces_free': 87,
 'total_zonespaces_blu': 285,
 'total_zonespaces_blu_free': 87,
 'total_zonespaces_carico_scarico': 0,
 'total_zonespaces_carico_scarico_free': 0,
 'total_zonespaces_disabili': 0,
 'total_zonespaces_disabili_free': 0,
 'total_parks': 9,
 'total_ciclobox': 9,
 'total_zones': 8}

In [73]:
# Scrittura su file JSON
with open("docs/data/descriptions.json", "w") as f:
    json.dump(descriptions, f, indent=4)

In [ ]:
data_parks = {}
data_parks["xAxis"] = {"type": "value"}
data_parks['yAxis'] = {"type": "category", "data": parks[parks['type'] == 'park'].name.unique().tolist()}
data_parks['series'] = [{"data": parks[(parks['type'] == 'park') & (parks.currentTimestamp == parks.currentTimestamp.max())].freeslots.tolist(), "type": "bar", "showBackground": True, "backgroundStyle": {"color": "rgba(180, 180, 180, 0.2)"}}]
# Definizione del dizionario che rappresenta il JSON

# Salva il JSON in un file
with open("docs/data/data_parks.json", "w", encoding="utf-8") as f:
    json.dump(data_parks, f, indent=2, ensure_ascii=False)

print("File JSON salvato con successo!")
